In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from py.registration import covers_bounding_box, get_warped_corners

# Create image containing a white square with blue background
def create_image(size=256):
    img = np.zeros((size, size, 3), dtype=np.uint8)
    img[:, :, 2] = 255
    start = int(size/4)
    end = int(size*3/4)
    img[start:end, start:end] = 255
    return img

# Create random transformation matrix
def create_random_transformation_matrix(angle_range=(-np.pi*0, np.pi*0), tx_range=(0,0), ty_range=(0,0), sx_range=(1,1), sy_range=(1,1)):
    # Random rotation angle
    angle = np.random.uniform(*angle_range)
    # Random translation
    tx = np.random.uniform(*tx_range)
    ty = np.random.uniform(*ty_range)
    # Random scaling
    sx = np.random.uniform(*sx_range)
    sy = np.random.uniform(*sy_range)
    # Create transformation matrix
    M = np.array([[sx*np.cos(angle), -sy*np.sin(angle), tx],
                  [sx*np.sin(angle),  sy*np.cos(angle), ty],
                  [0, 0, 1]])
    return M

def center_crop_img(img, size):
    """Center crop image to size"""
    h, w = img.shape[:2]
    y1 = (h - size) // 2
    y2 = y1 + size
    x1 = (w - size) // 2
    x2 = x1 + size
    return img[y1:y2, x1:x2]

# Apply random transformation to image
def apply_random_transformation(img, M, size=None):
    if not size:
        size = img.shape[0]

    # Apply transformation to image, with origin at center
    # Create translation matrix to move origin to center
    T = np.array([[1, 0, img.shape[1]/2],
                  [0, 1, img.shape[0]/2],
                  [0, 0, 1]])
    # Create inverse translation matrix
    T_inv = np.array([[1, 0, -img.shape[1]/2],
                      [0, 1, -img.shape[0]/2],
                      [0, 0, 1]])
    # Apply transformation
    # M = T_inv.dot(M).dot(T)
    img = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    
    return center_crop_img(img, size)

In [ ]:
# Create image
img = create_image()
M = create_random_transformation_matrix(
    angle_range=(-np.pi*0.5, np.pi*0.5),
    # tx_range=(-0.1, 0.1),
    # ty_range=(-0.1, 0.1),
    # sx_range=(0.9, 1.1),
    # sy_range=(0.9, 1.1),
)

# Print if image doesn't cover bounding box
bounding_box_size = (img.shape[1]//2, img.shape[0]//2)
if not covers_bounding_box(img, M, bounding_box_size, plotting=True):
    print('Image does not cover bounding box')

# Plot transformed image
img_warped = apply_random_transformation(img, M, size=bounding_box_size[0])
plt.imshow(img_warped)

# # Add warped corners to image in red
# warped_corners = get_warped_corners(img, M)
# for corner in warped_corners:
#     cv2.circle(img, tuple(corner.astype(int)), 5, (255, 0, 0), -1)

# # Add bounding box to image in green
# central_w, central_h = bounding_box_size
# central_x = (img.shape[1] - central_w) // 2
# central_y = (img.shape[0] - central_h) // 2
# cv2.rectangle(img, (central_x, central_y), (central_x + central_w, central_y + central_h), (0, 255, 0), 2)

# # Add wc_lines to image in purple
# wc_lines = [(warped_corners[i], warped_corners[(i + 1) % 4]) for i in range(4)]
# for line in wc_lines:
#     cv2.line(img, tuple(line[0].astype(int)), tuple(line[1].astype(int)), (255, 0, 255), 2)

# # Plot image
# plt.imshow(img)

In [ ]:
from py.helpers import get_sub_areas, get_area, concat_one, get_all_annotated_cases
from py.registration import get_align_transform, evaluate_registration, align
from wholeslidedata.annotation.structures import Polygon
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
from wholeslidedata.image.wholeslideimage import WholeSlideImage

def plot_compare_sub_areas(wsi1, wsi2, annotations1, annotations2, spacing=2.0, realign_true=True):
    """1 should be the p53, as that is the one with biopsy outlines"""
    outlines1 = [a.coordinates for a in annotations1 if isinstance(a, Polygon)]
    outlines2 = [a.coordinates for a in annotations2 if isinstance(a, Polygon)]
    scale = 1
    transform,_,_ = get_align_transform(wsi1, wsi2, outlines1[0], outlines2[0], spacing=8.0)

    # Plot biopsy
    fig, ax_biop = plt.subplots(len(outlines1), 3, figsize=(20,len(outlines1)*4))
    fig, ax = plt.subplots(len(outlines1), 3, figsize=(15,len(outlines1)*5))

    for i, biopsy in enumerate(outlines1):
        for j, wsi in enumerate([wsi1, wsi2]):
            padding = 0
            if j == 1 and realign_true:
                padding = 100

            biopsy_area = get_area(biopsy, spacing)
            sub_areas = get_sub_areas(biopsy_area, padding=padding)
            sub_area_row = sub_areas[int(len(sub_areas)//2)]    # middle row
            sub_area  = sub_area_row[int(len(sub_area_row)//2)] # middle of middle row
            
            sub_patch = wsi.get_patch(*sub_area, spacing)

            if j == 1:
                if realign_true:
                    sub_patch,msg,det = align(sub_patch, sub_patch_prev)
                metrics = evaluate_registration(sub_patch, [sub_patch_prev])
                metrics = [m[0] for m in metrics]
            else:
                msg = ""
                det = 0

            ax_biop[i,0].imshow(wsi.get_patch(*biopsy_area, spacing), alpha=1-j/2)
            ax_biop[i,j+1].imshow(wsi.get_patch(*biopsy_area, spacing))
            ax_biop[i,j+1].axis("Off")

            ax[i,0].imshow(sub_patch, alpha=1-j/2)
            ax[i,j+1].imshow(sub_patch)
            ax[i,j+1].axis("Off")

            biopsy = (transform @ concat_one(biopsy*scale).T).T / scale
            sub_patch_prev = sub_patch.copy()

        ax[i,0].text(50,50,f"{msg}, {det:.2f}\n{[round(m,2) for m in metrics]}")
        ax[i,0].axis("Off")
        ax_biop[i,0].axis("Off")

casename = "ASL-0049"
# casename = "RASL-20"
case = get_all_annotated_cases()[casename]
wsi = {"p53": WholeSlideImage(case["p53"]["wsi"]),
        "HE": WholeSlideImage(case["HE" ]["wsi"])}
wsa = {"p53": WholeSlideAnnotation(case["p53"]["wsa"]),
        "HE": WholeSlideAnnotation(case["HE" ]["wsa"])}
plot_compare_sub_areas(wsi['HE'], wsi['p53'], wsa['HE'].annotations, wsa['p53'].annotations, spacing=2.0, realign_true=True)